The first step is to prepare the dataset in the pytorch Dataset format.
For example like this with COCO annotation format:

In [ ]:
from torchvision.transforms import ToTensor

from core.architecture.datasets.object_detection_datasets import COCODataset

train_dataset_path = '' # your path to train part of dataset
val_dataset_path = '' # your path to validation part of dataset
train_json_path = '' # your path to train annotations of dataset
val_json_path = '' # your path to validation annotations of dataset

transform = ToTensor()
train_dataset = COCODataset(images_path=train_dataset_path, json_path=train_json_path, transform=transform)
val_dataset = COCODataset(images_path=val_dataset_path, json_path=val_json_path, transform=transform)

The second step is training the model.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")
from core.architecture.experiment.CVModule import FasterRCNNExperimenter

experimenter = FasterRCNNExperimenter(
    dataset_name='dataset', # Name of your dataset
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_classes=len(train_dataset.classes) + 1,
    structure_optimization='SVD',
    structure_optimization_params={'energy_thresholds': [0.99]}
)
experimenter.fit(100)

Now you can get predictions for all images in a folder. The method returns a dictionary {'image name': predictions}

In [ ]:
experimenter.predict('') # Path to your folder with images